In [1]:
import numpy as np
import tensorflow as tf

# tensorflow-datasetsモジュールを使ってMNISTデータセットをインポートしていきます。まだインストールしていない場合は以下のコマンドでインストールしましょう。
# pip install tensorflow-datasets 
# conda install tensorflow-datasets

import tensorflow_datasets as tfds

# このデータセットは、C:\Users\*USERNAME*\tensorflow_datasets\...に保存されます。

In [2]:
# tfds.loadを使ってデータセットを読み込んでいきます 
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# with_info=True をすることによって、データに関する情報を得ることができるようになります。

# as_supervised=Trueをすることによって、データを入力とターゲットの二つののタプル形式で作成することができま
print(mnist_info)

tfds.core.DatasetInfo(
    name='mnist',
    version=1.0.0,
    description='The MNIST database of handwritten digits.',
    urls=['https://storage.googleapis.com/cvdf-datasets/mnist/'],
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)



In [3]:
# 訓練用データとテスト用データをそれぞれの変数に代入します
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# ここから検証用データセットを作っていきます。まずは、検証用データに割り当てる割合を決めます。
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# 数字を整数に変換していきます（既に整数になっている場合も念のため行います）
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# テストデータの数も変数に入れていきます
num_test_samples = mnist_info.splits['test'].num_examples
# 数字を整数に変換していきます
num_test_samples = tf.cast(num_test_samples, tf.int64)


# 元の入力データの範囲である0から255を0と1に変えていきます
# 今回は、変換するための関数を作成していきます
def scale(image, label):
    # 数字が小数であることを確認します
    image = tf.cast(image, tf.float32)
    # 0と1の範囲におさめるため、255で割っていきます 
    image /= 255.

    return image, label

# mapメソッドを使ってデータの変換を行っていきます
scaled_train_and_validation_data = mnist_train.map(scale)

# テストデータに関しても同様の変換を行っていきます
test_data = mnist_test.map(scale)



In [4]:
#データをシャッフルするためのバッファのサイズを決めていきます
BUFFER_SIZE = 10000

# シャッフルメソッドを使ってデータをシャッフルしていきます
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# takeメソッドを使ってシャッフルした検証用データを変数に代入していきます
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

# skipメソッドを使って訓練用データセットを変数に代入します
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# batchの大きさを定義します
BATCH_SIZE = 100

# 訓練データをバッチの数毎に分けていきます
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

# テストデータに関してもバッチ毎に分けていきます
test_data = test_data.batch(num_test_samples)


# バリデーションデータセットに関し、入力とターゲットそれぞれの変数を作成していきます
validation_inputs, validation_targets = next(iter(validation_data))
print(validation_data )


<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>


In [5]:
input_size = 784
output_size = 10
# 隠れ層のユニットの数を定義します
hidden_layer_size = 50
    
# モデルの定義を進めていきます
model = tf.keras.Sequential([
    
    # 入力層の定義をしていきます
    # もとの行列のデータをベクトルに変換します
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # 入力層
    
    # tf.keras.layers.Dense は線形結合をするための計算を行います。つまり、入力と重みの積にバイアスを追加する形です
    # 引数として、ここでは隠れ層の数と活性化関数を指定します
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1番目の隠れ層
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2番目の隠れ層
    
    # 最後の層の活性関数はソフトマックスとします
    tf.keras.layers.Dense(output_size, activation='softmax') # 出力層
])

In [6]:
# 最適化アルゴリズムはadamを、損失関数はクロスエントロピーを使っていきます
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
# 繰り返しの回数を決めます
NUM_EPOCHS = 5
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# モデルに対してデータを入れて演算を行っていきます
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2,callbacks=[early_stopping])

Epoch 1/5
540/540 - 3s - loss: 0.4227 - accuracy: 0.8827 - val_loss: 0.2031 - val_accuracy: 0.9422
Epoch 2/5
540/540 - 3s - loss: 0.1714 - accuracy: 0.9495 - val_loss: 0.1492 - val_accuracy: 0.9603
Epoch 3/5
540/540 - 3s - loss: 0.1279 - accuracy: 0.9622 - val_loss: 0.1291 - val_accuracy: 0.9637
Epoch 4/5
540/540 - 3s - loss: 0.1044 - accuracy: 0.9690 - val_loss: 0.1072 - val_accuracy: 0.9695
Epoch 5/5
540/540 - 3s - loss: 0.0868 - accuracy: 0.9738 - val_loss: 0.0969 - val_accuracy: 0.9715


In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 10ms/step - loss: 0.1133 - accuracy: 0.9664


In [ ]:
# 出力結果の見た目を整えます
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.11. Test accuracy: 96.64%
